## Google Drive setting

In [259]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [260]:
%cd /content/drive/MyDrive/AIncome

/content/drive/MyDrive/AIncome


In [261]:
!pwd

/content/drive/MyDrive/AIncome


## Colab


In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 / 60 / 60 "h)"}'

In [ ]:
"""
function ClickConnect(){

console.log("Working3");
document.querySelector("#comments > span").click()
}
setInterval(ClickConnect,5000)
"""

### 作業フォルダ

In [262]:
!mkdir AutoTrade11

mkdir: cannot create directory ‘AutoTrade11’: File exists


In [263]:
%cd  AutoTrade11

/content/drive/MyDrive/AIncome/AutoTrade11


In [264]:
!pwd

/content/drive/MyDrive/AIncome/AutoTrade11


## 必要パッケージのインストール



### 強化学習 基礎関係

In [265]:
!pip install "gym==0.19.0"
!pip install stable-baselines[mpi]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 深層学習関係

In [266]:
!pip uninstall -y tensorflow-gpu
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0

Found existing installation: tensorflow-gpu 1.14.0
Uninstalling tensorflow-gpu-1.14.0:
  Successfully uninstalled tensorflow-gpu-1.14.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow_gpu-1.14.0-cp37-cp37m-manylinux1_x86_64.whl (377.1 MB)


### 強化学習 取引関係

In [267]:
!pip install gym-anytrading

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 取引分析関係

In [268]:
!pip install QuantStats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Tensorboard関係

In [269]:
!pip uninstall tensorboard-plugin-wit --yes

## インポート


In [270]:

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

import gym
import gym_anytrading
from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions
from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL

from stable_baselines.bench import Monitor
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import PPO2
from stable_baselines import ACKTR
from stable_baselines import A2C

import quantstats as qs

## 設定

In [311]:

# ログフォルダの生成
log_dir = './logs/'
os.makedirs(log_dir, exist_ok=True)


# train data
idx1 = 100
idx2 = 5000

# test data
idx3 = 6000

window_size = 100

trade_fee = 0

env_num = 100

total_timesteps=100000000


## Create datasets

### Read dataset

ビットコインのデータを読み込んでいきます．

In [312]:
df_dataset = pd.read_csv('./datasets/btc_1M.csv', header=None)

列名を表記します．

In [313]:
df_dataset.columns = ["Time", "Open", "High", "Low", "Close", "Vol"]
df_dataset["Time"] = pd.to_datetime(df_dataset["Time"])
df_dataset = df_dataset.set_index("Time")
print(df_dataset)

                        Open     High      Low    Close       Vol
Time                                                             
2022-08-21 15:29:00  2928990  2928991  2926123  2927317  1.156850
2022-08-21 15:30:00  2928563  2928693  2928326  2928326  0.195000
2022-08-21 15:31:00  2929395  2929395  2929395  2929395  0.008000
2022-08-21 15:32:00  2930299  2932254  2930299  2931734  0.525785
2022-08-21 15:33:00  2932041  2932041  2932041  2932041  0.008000
...                      ...      ...      ...      ...       ...
2022-08-26 00:27:00  2948026  2948428  2948025  2948390  0.315074
2022-08-26 00:28:00  2947000  2947000  2945053  2945707  0.191000
2022-08-26 00:29:00  2945707  2946000  2944025  2944025  0.519900
2022-08-26 00:31:00  2944000  2944000  2944000  2944000  0.031000
2022-08-26 00:32:00  2944000  2944000  2943012  2943012  0.013000

[6000 rows x 5 columns]


### Split data

In [314]:
df_train = df_dataset[0:int(len(df_dataset)*0.8)]

In [315]:
df_train

,Open,High,Low,Close,Vol
Time,,,,,
2022-08-21 15:29:00,2928990,2928991,2926123,2927317,1.156850
2022-08-21 15:30:00,2928563,2928693,2928326,2928326,0.195000
2022-08-21 15:31:00,2929395,2929395,2929395,2929395,0.008000
2022-08-21 15:32:00,2930299,2932254,2930299,2931734,0.525785
2022-08-21 15:33:00,2932041,2932041,2932041,2932041,0.008000
...,...,...,...,...,...
2022-08-25 03:03:00,2947842,2948094,2947842,2947887,0.345039
2022-08-25 03:04:00,2947887,2947887,2946885,2946885,3.061700
2022-08-25 03:05:00,2947882,2947882,2947475,2947475,0.090000


In [316]:
df_test = df_dataset[int(len(df_dataset)*0.8):]

In [317]:
df_test

,Open,High,Low,Close,Vol
Time,,,,,
2022-08-25 03:08:00,2944986,2945360,2944986,2945113,0.175146
2022-08-25 03:09:00,2944959,2946790,2944599,2946790,2.385007
2022-08-25 03:10:00,2946916,2949000,2946550,2948564,4.104590
2022-08-25 03:11:00,2948564,2949130,2948564,2949130,0.827563
2022-08-25 03:12:00,2948859,2949130,2948631,2949129,0.975001
...,...,...,...,...,...
2022-08-26 00:27:00,2948026,2948428,2948025,2948390,0.315074
2022-08-26 00:28:00,2947000,2947000,2945053,2945707,0.191000
2022-08-26 00:29:00,2945707,2946000,2944025,2944025,0.519900


`index`をdatetime型の時間にします．

## 環境の設定/生成

### データ読み込み部分

In [318]:
#####################################
# データ読み込み部分
#
def my_process_data(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]

    # -----------------------------
    # 特徴量生成
    #
    ohlc_features = env.df.loc[:, ['Close', 'Open', 'High', 'Low', 'Vol']].to_numpy()[start:end]

    #print(ohlc_features.shape)
    #print(np.diff(ohlc_features, axis=0).shape)
    diff1 = np.insert(np.diff(ohlc_features, axis=0), 0, 0, axis=0)
    diff2 = np.insert(np.diff(diff1, axis=0), 0, 0, axis=0)
    #print(diff)
    #signal_features = env.df.loc[:, ['Close', 'Open', 'High', 'Low', 'Vol']].to_numpy()[start:end]

    #signal_features = np.column_stack((ohlc_features, diff1, diff2))
    signal_features = np.column_stack((ohlc_features, ))

    print(">>> signal_features.shape")
    print( signal_features.shape)

    return prices, signal_features

### 環境クラス定義

In [319]:
#####################################
# 環境クラス
#
class MyBTCEnv(ForexEnv):
    _process_data = my_process_data

### 環境の生成

In [320]:
len(df_train)

4800

In [321]:
#####################################
# 環境の生成
#
env_marker_train = lambda:  MyBTCEnv(df=df_train, window_size=window_size, frame_bound=(window_size, len(df_train)))
env_marker_train.trade_fee = trade_fee

env_marker_test = lambda:  MyBTCEnv(df=df_test, window_size=window_size, frame_bound=(window_size, len(df_test)))
env_marker_test.trade_fee = trade_fee


env = DummyVecEnv([env_marker_train for _ in range(env_num)])

#env = Monitor(env, log_dir, allow_early_resets=True)

>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4800, 5)
>>> signal_features.shape
(4

## Train trade

In [ ]:
policy_kwargs = dict(net_arch=[64, 'lstm', dict(vf=[128, 128, 128], pi=[64, 64])])
model = PPO2('MlpLstmPolicy', env, verbose=1, policy_kwargs=policy_kwargs, nminibatches=1, tensorboard_log=log_dir)

time_start = time.time()
model.learn(total_timesteps=total_timesteps)

# 時間計測終了
time_end = time.time()
# 経過時間（秒）
tim = time_end- time_start

print("----------------")
print(tim)

ストリーミング出力は最後の 5000 行に切り捨てられました。
| n_updates          | 1737               |
| policy_entropy     | 0.6893289          |
| policy_loss        | -1.7376244e-06     |
| serial_timesteps   | 222336             |
| time_elapsed       | 9.26e+03           |
| total_timesteps    | 22233600           |
| value_loss         | 1673716100000000.0 |
-------------------------------------------
-------------------------------------------
| approxkl           | 2.8438851e-09      |
| clipfrac           | 0.0                |
| explained_variance | 0                  |
| fps                | 2409               |
| n_updates          | 1738               |
| policy_entropy     | 0.68930113         |
| policy_loss        | 2.774596e-07       |
| serial_timesteps   | 222464             |
| time_elapsed       | 9.27e+03           |
| total_timesteps    | 22246400           |
| value_loss         | 1501081700000000.0 |
-------------------------------------------
------------------------------------------
|

## Save model


In [ ]:
model.save("ppo2_forex-v0_lstm")

## Load model

In [ ]:
model = PPO2.load("ppo2_forex-v0_lstm")

## Test  trade

In [ ]:
import numpy as np

# env = DummyVecEnv([env_maker for _ in range(10)])
# env = env_maker()
env = env_marker_test()
observation = env.reset()

#env = env_marker_train()
#observation = env.reset()


while True:
    # observation = observation[np.newaxis, ...]
    observation = np.tile(observation, (env_num, 1, 1))

    # ----------------------------
    # observation
    #
    # print(observation)
    # print(observation.shape)

    # action = env.action_space.sample()
    action, _states = model.predict(observation)
    # print(action.shape)
    # print("------------------")
    # print(np.mean(action))
    action_mean = 1 if np.mean(action) >= 0.5 else 0
    # print(action_mean)
    observation, reward, done, info = env.step(action_mean)

    # print(done)
    # env.render()
    if done:
        print("info:", info)
        break

## Analysis Using quantstats

ポートフォリオのプロファイリングを行い，詳細な分析とリスクメトリクスを算出します．

In [ ]:
print(env)
print(len(env.history['total_profit']))
print(len(df_dataset.index))
print(len(df_dataset.index[idx1:idx2]))

In [ ]:
"""
qs.extend_pandas()

net_worth = pd.Series(env.history['total_profit'], index=df_test.index[idx1+1:idx2])
returns = net_worth.pct_change().iloc[1:]

# qs.reports.full(returns)
qs.reports.html(returns, output='a2c_quantstats.html')
"""

## Plot Results

In [ ]:
plt.figure(figsize=(30, 10))
env.render_all()
plt.show()

# Tensorboard

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/